In [9]:
from importlib import reload
import json
import numpy as np
import time
from env import VNLAExplorationBatch
import os
from collections import defaultdict
import utils
import math
import sys
from sys import getsizeof
from memory_profiler import profile

In [7]:
os.environ['PT_DATA_DIR']='/home/hoyeung/blob_matterport3d/'

nav_actions = ['left', 'right', 'up', 'down', 'forward', '<end>', '<start>', '<ignore>']
env_actions = [
        (0,-1, 0), # left   30 deg
        (0, 1, 0), # right  30 deg
        (0, 0, 1), # up     30 deg
        (0, 0,-1), # down   30 deg
        (1, 0, 0), # forward to navigable location at ix 1
        (0, 0, 0), # <end>
        (0, 0, 0), # <start>
        (0, 0, 0)  # <ignore>
    ]

env = VNLAExplorationBatch(nav_actions, env_actions, obs=None, batch_size=len(all_tr_scans))

In [8]:
scan_viewpt_map_all_tr_data = defaultdict(list)

for scan in all_tr_scans:
    scan_G = utils.load_nav_graphs(scan)
    for n in scan_G.nodes:
        scan_viewpt_map_all_tr_data[scan].append(n)

In [47]:
from memory_profiler import profile

In [49]:
size_list = []
deep_size_list = []

for (i, scan) in enumerate(all_tr_scans):
    n = np.random.choice(scan_viewpt_map_all_tr_data[scan])
    heading = (np.random.random() * 2 - 1) * 2 * math.pi
    elevation = (np.random.random() * 2 - 1) * 2 * math.pi
    env.sims[i].newEpisode(scan, n, heading, elevation) 
    size_list.append(getsizeof(env.sims[i]))
    deep_size_list.append(get_size(env.sims[i], set()))

In [12]:
@profile
def monitor_size(scan_list):
    for (i, scan) in enumerate(all_tr_scans):    
        n = np.random.choice(scan_viewpt_map_all_tr_data[scan])
        heading = (np.random.random() * 2 - 1) * 2 * math.pi
        elevation = (np.random.random() * 2 - 1) * 2 * math.pi
        env.sims[i].newEpisode(scan, n, heading, elevation) 

In [13]:
monitor_size(all_tr_scans)

ERROR: Could not find file <ipython-input-12-8cf510c630e5>
NOTE: %mprun can only be used on functions defined in physical files, and not in the IPython environment.


In [6]:
# 1 sim 1 scan
# 56s for 56 scans
all_tr_scans = ['ac26ZMwG7aT',
            'VzqfbhrpDEA',
            'vyrNrziPKCB',
            'mJXqzFtmKg4',
            'Vvot9Ly1tCj',
            'gTV8FGcVJC9',
            '1pXnuDYAj8r',
            'cV4RVeZvu5T',
            'p5wJjkQkbXX',
            '2n8kARJN3HM',
            'EDJbREhghzL',
            '8WUmhLawc2A',
            'SN83YJsR3w2',
            'JmbYfDe2QKZ',
            '759xd9YjKW5',
            'uNb9QFRL6hY',
            'JeFG25nYj2p',
            'qoiz87JEwZ2',
            'PX4nDJXEHrG',
            'D7N2EKCX4Sj',
            'aayBHfsNo7d',
            'b8cTxDM8gDG',
            '82sE5b5pLXE',
            '5LpN3gDmAk7',
            '1LXtFkjw3qL',
            'kEZ7cmS4wCh',
            'ULsKaCPVFJR',
            'ZMojNkEp431',
            'e9zR4mvMWw7',
            'VFuaQ6m2Qom',
            'rPc6DW4iMge',
            'S9hNv5qa7GM',
            '5q7pvUzZiYa',
            'pRbA3pwrgk9',
            'E9uDoFAP3SH',
            '17DRP5sb8fy',
            'XcA2TqTSSAj',
            'i5noydFURQK',
            '7y3sRwLe3Va',
            'sT4fr6TAbpF',
            'r47D5H71a5s',
            'jh4fc5c5qoQ',
            'r1Q1Z4BcV1o',
            'Uxmj2M2itWa',
            'VVfe2KiqLaN',
            'ur6pFq6Qu1A',
            'B6ByNegPMKs',
            'JF19kD82Mey',
            'sKLMLpTHeUy',
            '29hnd4uzFmX',
            'HxpKQynjfin',
            'V2XKFyX4ASd',
            'VLzqgDo317F',
            's8pcmisQ38h',
            'GdvgFV5R1Z5',
            'Pm6F8kyY3z2']

In [4]:
import sys

def get_size(obj, seen=None):
    """Recursively finds size of objects"""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size